<a href="https://colab.research.google.com/github/akim1337/sql/blob/main/leetcode_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
con = sqlite3.connect('db')

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

# 176 Вторая наибольшая зарплата
Вывести вторую наибольшую зарплату, если таковой нет, вывести null. Дополню данные, чтобы решение было более прозрачным

In [ ]:
data = [[1, 100],[2, 200],[3, 490],[5,200],[6, 230],[3, 1555]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [ ]:
employee.to_sql('employee',con,index=False,if_exists='replace')

6

In [ ]:
sql ='''
SELECT
    CASE
        WHEN COUNT(*) >= 1 THEN MAX(salary)
        ELSE NULL
    END AS SecondHighestSalary
FROM Employee
WHERE salary < (SELECT MAX(salary) FROM Employee)

'''

In [ ]:
select(sql)

,SecondHighestSalary
0,490


проще вариант

In [ ]:
sql ='''
select max(salary) as SecondHighestSalary
from Employee where salary < (select max(salary) from Employee)
'''

In [ ]:
select(sql)

,SecondHighestSalary
0,490


In [ ]:
# В Питоне
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    sorted_salaries = employee['salary'].sort_values(ascending=False).drop_duplicates()
    return pd.DataFrame({
        'SecondHighestSalary': [None if sorted_salaries.size < 2 else sorted_salaries.iloc[1]]
    })

In [ ]:
second_highest_salary(employee)

,SecondHighestSalary
0,490


# 182 Дубли емейлс
Напишите решение, чтобы сообщать обо всех повторяющихся электронных письмах. Обратите внимание, что гарантируется, что поле электронной почты не равно NULL.

Верните таблицу результатов в любом порядке.

Формат результата приведен в следующем примере.

In [ ]:
data = [[1, 'a@b.com'], [2, 'c@d.com'], [3, 'a@b.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'Int64', 'email':'object'})

In [ ]:
person.to_sql('person',con,index=False,if_exists='replace')

3

In [ ]:
sql ='''
select email
from (
SELECT email, count(email) as Email1
from person
group by email) t
where Email1 > 1
'''

In [ ]:
select(sql)

,email
0,a@b.com


или

In [ ]:
sql ='''
select email
from person
group by email
having count(email) > 1
'''

In [ ]:
select(sql)

,SecondHighestSalary
0,None


In [ ]:
# В Питоне
def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    results = pd.DataFrame()
    results = person.loc[person.duplicated(subset=['email']), ['email']]
    return results.drop_duplicates()


In [ ]:
duplicate_emails(person)

,email
2,a@b.com


# 175 Комбинация таблиц
Напишите решение, чтобы сообщить имя, фамилию, город и штат каждого человека в таблице Person. Если адрес идентификатора человека отсутствует в таблице Address, вместо этого сообщите null.

Верните таблицу результатов в любом порядке.

Формат результата приведен в следующем примере.

In [ ]:
data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})

data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

In [ ]:
person.to_sql('person',con,index=False,if_exists='replace')
address.to_sql('address',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
select firstName, lastName, city, state
from person left join address using(personId)
'''

In [ ]:
select(sql)

,firstName,lastName,city,state
0,Wang,Allen,None,None
1,Alice,Bob,New York City,New York


In [ ]:
# Питон

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
  return pd.merge(left=person, right=address, how='left', on='personId')[['firstName', 'lastName', 'city', 'state']]

combine_two_tables(person, address)

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


# 183 Пользователи которые ниразу не покупали
Напишите решение для поиска всех клиентов, которые никогда ничего не заказывают.

Верните таблицу результатов в любом порядке.

Формат результата приведен в следующем примере.

In [ ]:
data = [[1, 'Joe'], [2, 'Henry'], [3, 'Sam'], [4, 'Max']]
customers = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=['id', 'customerId']).astype({'id':'Int64', 'customerId':'Int64'})

In [ ]:
customers.to_sql('customers',con,index=False,if_exists='replace')
orders.to_sql('orders',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
select name as Customers
from customers left join orders ON customers.id = orders.customerId
WHERE customerId IS NULL
'''

In [ ]:
select(sql)

,Customers
0,Henry
1,Max


In [ ]:
# Питон
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
  merged_table = pd.merge(left=customers, right=orders, how='left', left_on='id', right_on='customerId')
  null_customers = merged_table['id_y'].isna()
  df_null_sustomers = merged_table[null_customers]
  df_null_sustomers = df_null_sustomers[['name']].rename(columns={'name':'Customers'})
  return df_null_sustomers


In [ ]:
find_customers(customers,orders)

,Customers
1,Henry
3,Max


# 181 Сотрудники зарабатывают Больше, чем Их Менеджеры
Напишите решение, чтобы найти сотрудников, которые зарабатывают больше, чем их менеджеры.

Верните таблицу результатов в любом порядке.

Формат результата приведен в следующем примере.

In [ ]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})

In [ ]:
employee.to_sql('employee',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
select name as Employee
from (
select t.id, t.name, t.salary, t.managerId, t2.salary as salary_mnr
from employee as t left join employee as t2 ON t.managerId = t2.id)t
where salary > salary_mnr


'''

In [ ]:
select(sql)

,Employee
0,Joe


In [ ]:
# Питон
def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
  merged_table = pd.merge(left=employee, right=employee, how='left', left_on='managerId', right_on='id')
  merged_table_filter = merged_table['salary_x'] > merged_table['salary_y']
  final_t = merged_table[merged_table_filter]
  Employee  = final_t[['name_x']].rename(columns={'name_x': 'Employee'})
  return Employee

In [ ]:
find_employees(employee)

,Employee
0,Joe


# 178 Ранговые баллы
Напишите решение, чтобы найти ранг баллов. Рейтинг должен быть рассчитан в соответствии со следующими правилами:

Баллы должны быть ранжированы от самого высокого к самому низкому.
Если есть связь между двумя баллами, оба должны иметь одинаковый рейтинг.
После ничьей следующий номер рейтинга должен быть следующим последовательным целым значением. Другими словами, между рангами не должно быть пробелов.
Верните таблицу результатов, упорядоченную по количеству баллов в порядке убывания.

Формат результата приведен в следующем примере.

In [ ]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})

In [ ]:
scores.to_sql('scores',con,index=False,if_exists='replace')

6

In [ ]:
sql ='''
select score, dense_rank() over(order by score desc) as rank
from scores
order by score desc


'''

In [ ]:
select(sql)

,Employee
0,Joe


In [ ]:
# Питон
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
  scores['rank'] = scores['score'].rank(method='dense', ascending=False)
  final_t = scores.sort_values(by='rank')
  return final_t

In [ ]:
order_scores(scores)

,id,score,rank
2,3,4.0,1.0
4,5,4.0,1.0
3,4,3.85,2.0
1,2,3.65,3.0
5,6,3.65,3.0
0,1,3.5,4.0


# 184 Самая высокая зарплата в отделе
Напишите решение для поиска сотрудников с самой высокой зарплатой в каждом из отделов.

Верните таблицу результатов в любом порядке.

Формат результата приведен в следующем примере.

In [ ]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})

data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
employee.to_sql('employee',con,index=False,if_exists='replace')
department.to_sql('department',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
select Department, Employee, Salary
from (
select t.id, t.name as Employee, t.salary, t.departmentId, t2.name as Department, dense_rank() over(partition by t2.name order by salary desc) as salary_dep_rank
from employee as t left join department as t2 ON t.departmentId = t2.id) t
where salary_dep_rank = 1
'''

In [ ]:
select(sql)

,Department,Employee,salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000


In [ ]:
# Питон
# def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
t = pd.merge(left=employee, right=department, how='left', left_on='departmentId', right_on='id')
t['salary_dep_rank'] = t.groupby('name_y')['salary'].rank(ascending=False, method='dense')
tt = t['salary_dep_rank'] == 1
final1 = t[tt]
final = final1[['name_y', 'name_x', 'salary']]
final



,name_y,name_x,salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


# 185 Топ 3 Самых высоких зарплат в отделе
Руководители компании заинтересованы в том, чтобы узнать, кто зарабатывает больше всего денег в каждом из отделов компании. Высокооплачиваемый сотрудник в отделе - это сотрудник, зарплата которого входит в тройку лучших уникальных зарплат для этого отдела.

Напишите решение, чтобы найти сотрудников с высокими доходами в каждом из отделов.

In [ ]:
data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1], [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})

data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
employee.to_sql('employee',con,index=False,if_exists='replace')
department.to_sql('department',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
select Department, Employee, Salary
from (
select t.id, t.name as Employee, t.salary, t.departmentId, t2.name as Department, dense_rank() over(partition by t2.name order by salary desc) as salary_dep_rank
from employee as t left join department as t2 ON t.departmentId = t2.id) t
where salary_dep_rank <= 3
'''

In [ ]:
select(sql)

,Department,Employee,salary
0,IT,Max,90000
1,IT,Joe,85000
2,IT,Randy,85000
3,IT,Will,70000
4,Sales,Henry,80000
5,Sales,Sam,60000


In [ ]:
# Питон
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
  t = pd.merge(left=employee, right=department, how='left', left_on='departmentId', right_on='id')
  t['salary_dep_rank'] = t.groupby('name_y')['salary'].rank(ascending=False, method='dense')
  tt = t['salary_dep_rank'] <= 3
  final1 = t[tt]
  final = final1[['name_y', 'name_x', 'salary']]
  return final

In [ ]:
department_highest_salary(employee, department)

,name_y,name_x,salary
0,IT,Joe,85000
1,Sales,Henry,80000
2,Sales,Sam,60000
3,IT,Max,90000
5,IT,Randy,85000
6,IT,Will,70000


# 1890 Последний вход в систему в 2020 году
Напишите решение, чтобы сообщить о последнем входе в систему для всех пользователей в 2020 году. Не включайте пользователей, которые не входили в систему в 2020 году.

In [ ]:
data = [[6, '2020-06-30 15:06:07'], [6, '2021-04-21 14:06:06'], [6, '2019-03-07 00:18:15'], [8, '2020-02-01 05:10:53'], [8, '2020-12-30 00:46:50'], [2, '2020-01-16 02:49:50'], [2, '2019-08-25 07:59:08'], [14, '2019-07-14 09:00:00'], [14, '2021-01-06 11:59:59']]
logins = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})

In [ ]:
logins.to_sql('logins',con,index=False,if_exists='replace')

9

In [ ]:
sql ='''
select user_id, max(time_stamp) as last_stamp
from (
select user_id, time_stamp, EXTRACT(YEAR FROM time_stamp) AS year from logins) t
where year = 2020
group by user_id
'''

# решение верное, но sqllite который в colab не читает функцию EXTRACT, поэтому тут решение не работает. Проверку делаю по постгресу, там все ок

In [ ]:
# Питон

def latest_login(logins: pd.DataFrame) -> pd.DataFrame:
  logins['time_stamp'] = pd.to_datetime(logins['time_stamp'])
  logins['year'] = logins['time_stamp'].dt.year
  logins = logins[logins['year'] == 2020]
  logins_max_time = logins.groupby('user_id').agg(last_stamp=('time_stamp', 'max')).reset_index()
  return logins_max_time


In [ ]:
latest_login(logins)

,user_id,last_stamp
0,2,2020-01-16 02:49:50
1,6,2020-06-30 15:06:07
2,8,2020-12-30 00:46:50


# 180 Последовательные числа
Найдите все числа, которые появляются по крайней мере три раза подряд.

In [ ]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2], [8, 2], [9, 2], [10, 7], [11, 7], [12, 7]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})

In [ ]:
logs.to_sql('logs',con,index=False,if_exists='replace')

12

In [ ]:
sql ='''
select id as ConsecutiveNums
from (
select id, num, lead(num,3) over(partition by num order by id) as lead_3_in_row
from logs
order by id) t
where lead_3_in_row = 1
'''


In [ ]:
select(sql)

,ConsecutiveNums
0,1


In [ ]:
logs

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2
7,8,2
8,9,2


In [ ]:
# Питон
def consecutive_numbers(logs: pd.DataFrame) -> pd.DataFrame:
  logs['num_next'] = logs['num'].shift(-1)
  logs['num_prev'] = logs['num'].shift(1)
  row_nums = logs[(logs['num'] == logs['num_prev']) & (logs['num'] == logs['num_next'])]
  ConsecutiveNums1  = row_nums['num'].drop_duplicates()
  ConsecutiveNums = ConsecutiveNums1.count()
  return ConsecutiveNums


In [ ]:
ConsecutiveNums

AttributeError: ignored

# 584 Найдите рефери по работе с клиентами
Найдите имена клиентов, на которые не ссылается клиент с id = 2.

In [ ]:
data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype({'id':'Int64', 'name':'object', 'referee_id':'Int64'})

In [ ]:
customer.to_sql('customer',con,index=False,if_exists='replace')

6

In [ ]:
sql ='''
select name
from
(SELECT id, name, COALESCE(referee_id, 0) as referee_id
from customer) t
where referee_id not in (2)
'''

In [ ]:
select(sql)

,name
0,Will
1,Jane
2,Bill
3,Zack


In [ ]:
# Питон
def find_customer_referee(customer: pd.DataFrame) -> pd.DataFrame:
  t = customer.fillna(0)
  name = t[t['referee_id'] != 2][['name']]
  return name


In [ ]:
find_customer_referee(customer)

,name
0,Will
1,Jane
3,Bill
4,Zack


# 585 Инвестиции в 2016 году
Напишите решение для отчета о сумме всех общих инвестиционных значений за 2016 год tiv_2016 для всех страхователей, которые:
- имеют то же значение tiv_2015, что и один или несколько других страхователей
- находятся не в том же городе, что и любой другой страхователь (т.е. пары атрибутов (широта, долгота) должны быть уникальными)

Округлите tiv_2016 до двух знаков после запятой.

In [ ]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

In [ ]:
insurance.to_sql('insurance',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
WITH t_final as (
Select pid, tiv_2016
from (
select pid, tiv_2016, count(*) OVER(partition by lat,lon) as lat_lon_unq, count(tiv_2015) OVER(partition by tiv_2015) as not_unq
from insurance)
where lat_lon_unq = 1 and not_unq >= 2)

select ROUND(sum(tiv_2016),2) as tiv_2016
from t_final

'''


In [ ]:
select(sql)

,tiv_2016
0,45.0


In [ ]:
insurance

,pid,tiv_2015,tiv_2016,lat,lon
0,1,10.0,5.0,10.0,10.0
1,2,20.0,20.0,20.0,20.0
2,3,10.0,30.0,20.0,20.0
3,4,10.0,40.0,40.0,40.0


In [ ]:
# Питон
def find_investments(insurance: pd.DataFrame) -> pd.DataFrame:
  insurance['tiv_2015_cnt'] = insurance.groupby('tiv_2015')['tiv_2015'].transform('count')
  insurance['lat_cnt'] = insurance.groupby('lat')['lat'].transform('count')
  insurance['lon_cnt'] = insurance.groupby('lon')['lon'].transform('count')
  t = insurance[(insurance['tiv_2015_cnt'] >= 2) & ((insurance['lat_cnt'] == 1) & (insurance['lat_cnt'] == 1))]
  tiv_2016_sum = t['tiv_2016'].sum()
  return tiv_2016_sum

In [ ]:
find_investments(insurance)

NameError: ignored

# 626 Поменяйтесь местами

Напишите решение для замены идентификатора места каждых двух последовательных студентов. Если количество студентов нечетное, идентификатор последнего студента не меняется местами.

Верните таблицу результатов, упорядоченную по идентификатору в порядке возрастания.

In [ ]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})

In [ ]:
seat.to_sql('seat',con,index=False,if_exists='replace')

5

In [ ]:
sql ='''
SELECT id,
       CASE
         WHEN mod(id, 2) = 1 AND lead(id) OVER(ORDER BY id) IS NOT NULL THEN lead(student) OVER(ORDER BY id)
         WHEN mod(id, 2) = 0 THEN lag(student) OVER(ORDER BY id)
         ELSE student
       END AS student
FROM seat
ORDER BY id
'''
# Забрал готовое решение, не знал про mod(столбец, 2)


In [ ]:
select(sql)

,id,student
0,1,Doris
1,2,Abbot
2,3,Green
3,4,Emerson
4,5,Jeames


# 1045 Клиенты, которые купили все продукты

Напишите решение для отображения идентификаторов клиентов из таблицы клиентов, которые купили все продукты в таблице продуктов.

In [ ]:
data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6], [1, 0]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype({'customer_id':'Int64', 'product_key':'Int64'})
data = [[5], [6], [0]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key':'Int64'})

In [ ]:
customer.to_sql('customer',con,index=False,if_exists='replace')
product.to_sql('product',con,index=False,if_exists='replace')

3

In [ ]:
sql ='''
SELECT customer_id
from customer
group by customer_id
having count(distinct product_key) = (select count(distinct product_key) from Product)

'''

In [ ]:
select(sql)

,customer_id
0,1


In [ ]:
#Питон
def find_customers(customer: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
  unique_customer_ids = customer.groupby('customer_id')['product_key'].nunique()
  filtered_customer_ids = unique_customer_ids[unique_customer_ids == product['product_key'].nunique()]
  result_table = pd.DataFrame({'customer_id': filtered_customer_ids})
  return result_table


In [ ]:
find_customers(customer, product)

NameError: ignored

# 602 Запросы в друзья II: У кого больше всего друзей

Напишите решение, чтобы найти людей, у которых больше всего друзей, и наибольшее число друзей.

In [ ]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})

In [ ]:
request_accepted.to_sql('request_accepted',con,index=False,if_exists='replace')


4

In [ ]:
sql ='''
SELECT *
from request_accepted
'''

In [ ]:
select(sql)

,requester_id,accepter_id,accept_date
0,1,2,2016-06-03 00:00:00
1,1,3,2016-06-08 00:00:00
2,2,3,2016-06-08 00:00:00
3,3,4,2016-06-09 00:00:00


In [ ]:
sql ='''
SELECT t.id, count(t.id) as num
From (
SELECT requester_id as id
FROM RequestAccepted
UNION ALL
SELECT accepter_id
FROM RequestAccepted ) t
group by t.id
order by num desc
limit 1
'''

In [ ]:
select(sql)

,id,num
0,3,3


In [ ]:
#Питон
# def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
# return

def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([request_accepted["requester_id"], request_accepted["accepter_id"]]).\
            value_counts().\
            reset_index(name='num').\
            rename(columns={'index':'id'}).\
            head(1)


In [ ]:
most_friends(request_accepted)

,id,num
0,3,3


# 1070 Анализ продаж продукции III

Напишите решение для выбора идентификатора продукта, года выпуска, количества и цены за первый год каждого проданного продукта.

In [ ]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [ ]:
sales.to_sql('sales',con,index=False,if_exists='replace')
product.to_sql('product',con,index=False,if_exists='replace')

3

In [ ]:
sql ='''
SELECT *
from sales
'''

In [ ]:
select(sql)

,sale_id,product_id,year,quantity,price
0,1,100,2008,10,5000
1,2,100,2009,12,5000
2,7,200,2011,15,9000


In [ ]:
sql ='''
SELECT *
from product
'''

In [ ]:
select(sql)

,product_id,product_name
0,100,Nokia
1,200,Apple
2,300,Samsung


In [ ]:
sql ='''
SELECT t.id, count(t.id) as num
From (
SELECT requester_id as id
FROM RequestAccepted
UNION ALL
SELECT accepter_id
FROM RequestAccepted ) t
group by t.id
order by num desc
limit 1
'''

In [ ]:
select(sql)

,id,num
0,3,3


In [ ]:
#Питон
def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
  df = sales[sales['year'] == sales.groupby(['product_id'])['year'].transform('min')].rename(columns={'year': 'first_year'})
  return df


In [ ]:
sales_analysis(sales, product)

,sale_id,product_id,first_year,quantity,price
0,1,100,2008,10,5000
2,7,200,2011,15,9000


In [ ]:
sales['year'].value_counts()

NameError: ignored

In [ ]:
data = [['A'],['B'],['C'],['d']]
employee = pd.DataFrame(data, columns=['id_1']).astype({'id_1':'object'})
data1 = [['g'],['k']]
employee1 = pd.DataFrame(data1, columns=['id_2']).astype({'id_2':'object'})


In [ ]:
employee.to_sql('employee',con,index=False,if_exists='replace')
employee1.to_sql('employee1',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
SELECT *
from employee t inner join employee1 t1 ON t.id_1 = t1.id_2

'''

In [ ]:
select(sql)

,id_1,id_2


# 1068 Анализ продаж продукта I
Напишите решение для отображения product_name, year и price для каждого sale_id в таблице продаж.


In [ ]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [ ]:
sales.to_sql('sales',con,index=False,if_exists='replace')
product.to_sql('product',con,index=False,if_exists='replace')

3

In [ ]:
sql ='''
SELECT *
from sales
'''

In [ ]:
select(sql)

,sale_id,product_id,year,quantity,price
0,1,100,2008,10,5000
1,2,100,2009,12,5000
2,7,200,2011,15,9000


In [ ]:
sql ='''
SELECT *
from product
'''

In [ ]:
select(sql)

,product_id,product_name
0,100,Nokia
1,200,Apple
2,300,Samsung


In [ ]:
sql ='''
SELECT product_name, year, price
from sales left join product using(product_id)
'''

In [ ]:
select(sql)

,sale_id,product_name,year,price
0,1,Nokia,2008,5000
1,2,Nokia,2009,5000
2,7,Apple,2011,9000


# 1070 Анализ продаж продукции III
Напишите решение для выбора идентификатора продукта, года выпуска, количества и цены за первый год каждого проданного продукта.


In [ ]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [ ]:
sales.to_sql('sales',con,index=False,if_exists='replace')
product.to_sql('product',con,index=False,if_exists='replace')

3

In [ ]:
sql ='''
SELECT *
from sales
'''

In [ ]:
select(sql)

,sale_id,product_id,year,quantity,price
0,1,100,2008,10,5000
1,2,100,2009,12,5000
2,7,200,2011,15,9000


In [ ]:
sql ='''
select  t.product_id, t1.year as first_year , t1.quantity, t1.price
from
(SELECT min(year) as first_year_buy, product_id
from sales
group by product_id) t
left join sales t1 on t.product_id = t1.product_id and t.first_year_buy = t1.year
'''

In [ ]:
select(sql)

,product_id,first_year,quantity,price
0,100,2008,10,5000
1,200,2011,15,9000


# 1158 Анализ рынка I
Напишите решение, которое нужно найти для каждого пользователя, дату присоединения и количество заказов, которые он сделал в качестве покупателя в 2019 году.


In [ ]:
data = [[1, '2018-01-01', 'Lenovo'], [2, '2018-02-09', 'Samsung'], [3, '2018-01-19', 'LG'], [4, '2018-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})
data = [[1, '2019-08-01', 4, 1, 2], [2, '2018-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2018-08-04', 1, 4, 2], [5, '2018-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})

In [ ]:
users.to_sql('users',con,index=False,if_exists='replace')
orders.to_sql('orders',con,index=False,if_exists='replace')
items.to_sql('items',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
SELECT *
from users
'''

In [ ]:
select(sql)

,user_id,join_date,favorite_brand
0,1,2018-01-01 00:00:00,Lenovo
1,2,2018-02-09 00:00:00,Samsung
2,3,2018-01-19 00:00:00,LG
3,4,2018-05-21 00:00:00,HP


In [ ]:
sql ='''
SELECT *
from orders
'''

In [ ]:
select(sql)

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01 00:00:00,4,1,2
1,2,2018-08-02 00:00:00,2,1,3
2,3,2019-08-03 00:00:00,3,2,3
3,4,2018-08-04 00:00:00,1,4,2
4,5,2018-08-04 00:00:00,1,3,4
5,6,2019-08-05 00:00:00,2,2,4


In [ ]:
sql ='''
WITH t as (
  select buyer_id, strftime('%Y', order_date) AS year
  from orders
)

SELECT t1.user_id, t1.join_date, count(year) as orders
from users t1 left join t ON t1.user_id = t.buyer_id
group by t1.user_id, t1.join_date
'''

In [ ]:
select(sql)

,user_id,join_date,orders
0,1,2018-01-01 00:00:00,2
1,2,2018-02-09 00:00:00,2
2,3,2018-01-19 00:00:00,1
3,4,2018-05-21 00:00:00,1


In [ ]:
sql ='''
select user_id as buyer_id, join_date, count(seller_id) as orders_in_2019
from users u left join orders on u.user_id = orders.buyer_id and strftime('%Y', order_date) = '2019'

group by user_id, join_date
order by user_id
'''

In [ ]:
select(sql)

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01 00:00:00,1
1,2,2018-02-09 00:00:00,2
2,3,2018-01-19 00:00:00,0
3,4,2018-05-21 00:00:00,0


# 1164 Цена продукта на определенную дату
Напишите решение, чтобы найти цены на все продукты на 2019-08-16. Предположим, что цена всех продуктов до любого изменения равна 10.


In [ ]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})

In [ ]:
products.to_sql('products',con,index=False,if_exists='replace')


6

In [ ]:
sql ='''
SELECT *
from products
'''

In [ ]:
select(sql)

,product_id,new_price,change_date
0,1,20,2019-08-14 00:00:00
1,2,50,2019-08-14 00:00:00
2,1,30,2019-08-15 00:00:00
3,1,35,2019-08-16 00:00:00
4,2,65,2019-08-17 00:00:00
5,3,20,2019-08-18 00:00:00


In [ ]:
# select product_id, price
sql ='''
with t as (
SELECT *, CASE WHEN change_date <= '2019-08-16 00:00:00' THEN new_price ELSE 10 END as price
from products)

select *
from t
order by product_id

'''

In [ ]:
select(sql)

,product_id,new_price,change_date,price
0,1,20,2019-08-14 00:00:00,20
1,1,30,2019-08-15 00:00:00,30
2,1,35,2019-08-16 00:00:00,35
3,2,50,2019-08-14 00:00:00,50
4,2,65,2019-08-17 00:00:00,10
5,3,20,2019-08-18 00:00:00,10


In [ ]:
sql ='''
WITH cte AS
(SELECT *, RANK() OVER (PARTITION BY product_id ORDER BY change_date DESC) AS r
FROM Products
WHERE change_date<= '2019-08-16')

SELECT product_id, new_price AS price
FROM cte
WHERE r = 1
UNION ALL
SELECT product_id, 10 AS price
FROM Products
WHERE product_id NOT IN (SELECT product_id FROM cte)
'''

In [ ]:
select(sql)

,product_id,price
0,1,30
1,2,50
2,3,10


# 619 Самое большое одиночное число
Одиночное число - это число, которое появилось в таблице "Мои номера" только один раз. Найдите наибольшее одиночное число. Если одиночного числа нет, сообщите null.


In [ ]:
data = [[8], [8], [3], [3], [1], [4], [9], [5]]
my_numbers = pd.DataFrame(data, columns=['num']).astype({'num':'Int64'})

In [ ]:
my_numbers

,num
0,8
1,8
2,3
3,3
4,1
5,4
6,9
7,5


In [ ]:
my_numbers.to_sql('my_numbers',con,index=False,if_exists='replace')


8

In [ ]:
sql ='''
SELECT *
from my_numbers
order by num
'''

In [ ]:
select(sql)

,num
0,1
1,3
2,3
3,4
4,5
5,8
6,8
7,9


In [ ]:
sql ='''
select num
from my_numbers
group by num
having count(num) = 1
order by num desc
limit 1
'''

In [ ]:
select(sql)

,num
0,9


In [ ]:
def biggest_single_number(my_numbers: pd.DataFrame) -> pd.DataFrame:
    t = my_numbers.drop_duplicates(keep=False)
    if t.empty:
        return pd.DataFrame({'num': [np.NaN]})
    return pd.DataFrame({'num': [max(t['num'])]})

In [ ]:
biggest_single_number(my_numbers)

,num
0,9


# 620 Нескучные фильмы
Напишите решение, позволяющее сообщать о фильмах с нечетным идентификатором и описанием, которое не является "скучным".

Верните таблицу результатов, упорядоченную по рейтингу в порядке убывания.


In [ ]:
data = [[1, 'War', 'great 3D', 8.9], [2, 'Science', 'fiction', 8.5], [3, 'irish', 'boring', 6.2], [4, 'Ice song', 'Fantacy', 8.6], [5, 'House card', 'Interesting', 9.1]]
cinema = pd.DataFrame(data, columns=['id', 'movie', 'description', 'rating']).astype({'id':'Int64', 'movie':'object', 'description':'object', 'rating':'Float64'})

In [ ]:
cinema.to_sql('cinema',con,index=False,if_exists='replace')

5

In [ ]:
sql ='''
SELECT *
from cinema
WHERE id % 2 = 1 AND description NOT LIKE '%boring%'
order by rating desc
'''

In [ ]:
select(sql)

,id,movie,description,rating
0,5,House card,Interesting,9.1
1,1,War,great 3D,8.9


In [ ]:
def not_boring_movies(cinema: pd.DataFrame) -> pd.DataFrame:
    cinema = cinema[~cinema['description'].str.contains('boring')]
    cinema = cinema[cinema['id'] % 2 == 1].sort_values(by='rating', ascending=False)
    return cinema

In [ ]:
not_boring_movies(cinema)

,id,movie,description,rating
4,5,House card,Interesting,9.1
0,1,War,great 3D,8.9


# 197 Повышение температуры
Напишите решение, чтобы найти идентификатор всех дат с более высокими температурами по сравнению с предыдущими датами (вчера).


In [ ]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [ ]:
weather.to_sql('weather',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
SELECT *
from weather
'''

In [ ]:
select(sql)

,id,recordDate,temperature
0,1,2015-01-01 00:00:00,10
1,2,2015-01-02 00:00:00,25
2,3,2015-01-03 00:00:00,20
3,4,2015-01-04 00:00:00,30


In [ ]:
# postgres
sql ='''
SELECT w1.id
FROM Weather w1
JOIN Weather w2 ON w1.recordDate = (w2.recordDate + INTERVAL '1 DAY')
WHERE w1.temperature > w2.temperature;
'''

In [ ]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather.sort_values('recordDate', inplace=True)
    weather['recordDate_shifted'] = weather['recordDate'].shift(1)
    weather['temperature_shifted'] = weather['temperature'].shift(1)
    return weather[((weather['recordDate']-weather['recordDate_shifted']).dt.days == 1)  &  (weather['temperature']-weather['temperature_shifted']>0)][['id']]

In [ ]:
rising_temperature(weather)

,id
1,2
3,4


# 1341 Рейтинг фильма (не решено)
Найдите имя пользователя, который оценил наибольшее количество фильмов. В случае равенства верните лексикографически меньшее имя пользователя.

Найдите название фильма с самым высоким средним рейтингом в феврале 2020 года. В случае ничьей верните лексикографически меньшее название фильма.


In [ ]:
data = [[1, 'Avengers'], [2, 'Frozen 2'], [3, 'Joker']]
movies = pd.DataFrame(data, columns=['movie_id', 'title']).astype({'movie_id':'Int64', 'title':'object'})
data = [[1, 'Daniel'], [2, 'Monica'], [3, 'Maria'], [4, 'James']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})
data = [[1, 1, 3, '2020-01-12'], [1, 2, 4, '2020-02-11'], [1, 3, 2, '2020-02-12'], [1, 4, 1, '2020-01-01'], [2, 1, 5, '2020-02-17'], [2, 2, 2, '2020-02-01'], [2, 3, 2, '2020-03-01'], [3, 1, 3, '2020-02-22'], [3, 2, 4, '2020-02-25']]
movie_rating = pd.DataFrame(data, columns=['movie_id', 'user_id', 'rating', 'created_at']).astype({'movie_id':'Int64', 'user_id':'Int64', 'rating':'Int64', 'created_at':'datetime64[ns]'})

In [ ]:
movies.to_sql('movies',con,index=False,if_exists='replace')
users.to_sql('users',con,index=False,if_exists='replace')
movie_rating.to_sql('movie_rating',con,index=False,if_exists='replace')

9

In [ ]:
sql ='''
SELECT *
from movie_rating
'''

# 1280 Студенты и экзамены
Напишите решение, чтобы найти количество посещений каждым студентом каждого экзамена.

Верните таблицу результатов, упорядоченную по student_id и subject_name.


In [ ]:
data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [ ]:
students.to_sql('students',con,index=False,if_exists='replace')
subjects.to_sql('subjects',con,index=False,if_exists='replace')
examinations.to_sql('examinations',con,index=False,if_exists='replace')

11

In [ ]:
sql ='''
SELECT *
from examinations left join students using(student_id)
order by subject_name
'''

In [ ]:
select(sql)

,student_id,subject_name,student_name
0,1,Math,Alice
1,1,Math,Alice
2,13,Math,John
3,2,Math,Bob
4,1,Math,Alice
5,1,Physics,Alice
6,1,Physics,Alice
7,13,Physics,John
8,1,Programming,Alice
9,2,Programming,Bob


In [ ]:
sql ='''
SELECT s.student_id, s.student_name, sub.subject_name, COUNT(e.student_id) AS attended_exams
FROM Students s
CROSS JOIN Subjects sub
LEFT JOIN Examinations e ON s.student_id = e.student_id AND sub.subject_name = e.subject_name
GROUP BY s.student_id, s.student_name, sub.subject_name
ORDER BY s.student_id, sub.subject_name;
'''

In [ ]:
select(sql)

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Physics,0
5,2,Bob,Programming,1
6,6,Alex,Math,0
7,6,Alex,Physics,0
8,6,Alex,Programming,0
9,13,John,Math,1


# 607 Специалист по продажам
Напишите решение, чтобы найти имена всех продавцов, у которых не было заказов, связанных с компанией с названием "RED".


In [ ]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
sales_person = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})

In [ ]:
sales_person.to_sql('sales_person',con,index=False,if_exists='replace')
company.to_sql('company',con,index=False,if_exists='replace')
orders.to_sql('orders',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
SELECT sp.name
FROM orders
INNER JOIN sales_person sp ON sp.sales_id = orders.sales_id
INNER JOIN company ON company.com_id = orders.com_id
WHERE company.name NOT LIKE 'RED'
'''

In [ ]:
select(sql)

,name
0,Pam
1,Alex


In [ ]:
def sales_persons11(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    t = pd.merge(sales_person, orders, on='sales_id', how='inner')
    t1 = pd.merge(t, company, on='com_id', how='inner')
    t1 = t1.rename(columns={'name_x': 'name'})
    return t1[(~t1['name_y'].str.contains('RED'))][['name']]

In [ ]:
sales_persons11(sales_person, company, orders)

,name
2,Pam
3,Alex


# 1050 Актеры и режиссеры, Которые Сотрудничали Не менее Трех Раз
Напишите решение, чтобы найти все пары (actor_id, director_id), в которых актер сотрудничал с режиссером по крайней мере три раза


In [ ]:
data = [[1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 3], [1, 2, 4], [2, 1, 5], [2, 1, 6]]
actor_director = pd.DataFrame(data, columns=['actor_id', 'director_id', 'timestamp']).astype({'actor_id':'int64', 'director_id':'int64', 'timestamp':'int64'})

In [ ]:
actor_director.to_sql('actor_director',con,index=False,if_exists='replace')

7

In [ ]:
sql ='''
SELECT actor_id, director_id
from actor_director
group by actor_id, director_id
having count(timestamp) >=3
'''

In [ ]:
select(sql)

,actor_id,director_id
0,1,1


In [ ]:
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    filtered_df = actor_director[actor_director.groupby(['actor_id', 'director_id'])['timestamp'].transform('count') >= 3]
    filtered_df = filtered_df[['actor_id', 'director_id']].drop_duplicates()
    return filtered_df

In [ ]:
actors_and_directors(actor_director)

,actor_id,director_id
0,1,1


# 1075 Сотрудники проекта I
Напишите SQL-запрос, который сообщает средний стаж работы всех сотрудников по каждому проекту, округленный до 2 цифр.


In [ ]:
data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype({'project_id':'Int64', 'employee_id':'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype({'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'})

In [ ]:
project.to_sql('project',con,index=False,if_exists='replace')
employee.to_sql('employee',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
SELECT project_id, ROUND(avg(experience_years),2) as average_years
from project left join employee using(employee_id)
group by project_id
'''

In [ ]:
select(sql)

,project_id,average_years
0,1,2.0
1,2,2.5


In [ ]:
def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    t = pd.merge(project, employee, how='left', on='employee_id')
    t = t.groupby(['project_id']).agg(average_years=('experience_years', 'mean')).reset_index()
#   t = t.groupby('project_id')['experience_years'].mean().round(2).rename('average_years').reset_index()
    t['average_years'] = t['average_years'].round(2)
    return t

In [ ]:
project_employees_i(project, employee)

,project_id,average_years
0,1,2.0
1,2,2.5


# 1321 Рост ресторана
Вычислите скользящее среднее значение того, сколько клиент заплатил за семидневный период (т.е. текущий день + 6 дней до этого). значение average_amount должно быть округлено до двух знаков после запятой.


In [ ]:
data = [[1, 'Jhon', '2019-01-01', 100], [2, 'Daniel', '2019-01-02', 110], [3, 'Jade', '2019-01-03', 120], [4, 'Khaled', '2019-01-04', 130], [5, 'Winston', '2019-01-05', 110], [6, 'Elvis', '2019-01-06', 140], [7, 'Anna', '2019-01-07', 150], [8, 'Maria', '2019-01-08', 80], [9, 'Jaze', '2019-01-09', 110], [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype({'customer_id':'Int64', 'name':'object', 'visited_on':'datetime64[ns]', 'amount':'Int64'})

In [ ]:
customer.to_sql('customer',con,index=False,if_exists='replace')

11

In [ ]:
sql ='''
SELECT *
from customer
order by customer_id, name
'''

In [ ]:
select(sql)

,customer_id,name,visited_on,amount
0,1,Jhon,2019-01-01 00:00:00,100
1,1,Jhon,2019-01-10 00:00:00,130
2,2,Daniel,2019-01-02 00:00:00,110
3,3,Jade,2019-01-03 00:00:00,120
4,3,Jade,2019-01-10 00:00:00,150
5,4,Khaled,2019-01-04 00:00:00,130
6,5,Winston,2019-01-05 00:00:00,110
7,6,Elvis,2019-01-06 00:00:00,140
8,7,Anna,2019-01-07 00:00:00,150
9,8,Maria,2019-01-08 00:00:00,80


In [ ]:
sql ='''
SELECT
    visited_on,
    SUM(amount) OVER (ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as amount,
    ROUND(AVG(amount) OVER (ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW),2) as average_amount
FROM
    (SELECT
        visited_on,
        SUM(amount) amount
    FROM Customer
    GROUP BY visited_on
    )

'''
# OFFSET 6 в конце запроса

In [ ]:
select(sql)

,visited_on,amount,average_amount
0,2019-01-01 00:00:00,100,100.00
1,2019-01-02 00:00:00,210,105.00
2,2019-01-03 00:00:00,330,110.00
3,2019-01-04 00:00:00,460,115.00
4,2019-01-05 00:00:00,570,114.00
5,2019-01-06 00:00:00,710,118.33
6,2019-01-07 00:00:00,860,122.86
7,2019-01-08 00:00:00,840,120.00
8,2019-01-09 00:00:00,840,120.00
9,2019-01-10 00:00:00,1000,142.86


In [ ]:
# сложнааа в питоне
def restaurant_growth(customer: pd.DataFrame) -> pd.DataFrame:
    # Группировка и суммирование по дате посещения
    grouped_df = customer.groupby('visited_on').agg(amount=('amount', 'sum')).reset_index()

    # Вычисление суммы и среднего значения с окном 7 дней
    grouped_df['amount1'] = grouped_df['amount'].rolling(window=7, min_periods=1).sum()
    grouped_df['average_amount'] = grouped_df['amount'].rolling(window=7, min_periods=1).mean().round(2)

    # Пропуск первых 6 строк
    grouped_df = grouped_df.iloc[6:]
    grouped_df = grouped_df.rename(columns={'amount': 'amount_old', 'amount1': 'amount'})

    return grouped_df[['visited_on', 'amount', 'average_amount']]

In [ ]:
restaurant_growth(customer)

,visited_on,amount,average_amount
6,2019-01-07,860.0,122.86
7,2019-01-08,840.0,120.00
8,2019-01-09,840.0,120.00
9,2019-01-10,1000.0,142.86


# 1084 Анализ продаж III

Напишите решение для отчета о продуктах, которые были проданы только в первом квартале 2019 года. То есть в период с 2019-01-01 по 2019-03-31 включительно


In [ ]:
data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 2, 3, '2019-06-02', 1, 800], [3, 3, 4, '2019-05-13', 2, 2800], [2, 5, 2, '2019-02-19', 1, 1700]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})

In [ ]:
product.to_sql('product',con,index=False,if_exists='replace')
sales.to_sql('sales',con,index=False,if_exists='replace')

5

In [ ]:
sql ='''
SELECT *
from sales

'''

In [ ]:
select(sql)

,seller_id,product_id,buyer_id,sale_date,quantity,price
0,1,1,1,2019-01-21 00:00:00,2,2000
1,1,2,2,2019-02-17 00:00:00,1,800
2,2,2,3,2019-06-02 00:00:00,1,800
3,3,3,4,2019-05-13 00:00:00,2,2800
4,2,5,2,2019-02-19 00:00:00,1,1700


In [ ]:
sql ='''
SELECT product_id, product_name
FROM Sales left join Product Using(product_id)
GROUP BY product_id
HAVING MIN(sale_date) >= '2019-01-01' AND MAX(sale_date) <= '2019-03-31'

'''
# WHERE sale_date BETWEEN '2019-01-01' AND '2019-03-31'

In [ ]:
select(sql)

,product_id,product_name
0,1,S8
1,5,None


In [ ]:
def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    sales = sales.groupby('product_id')['sale_date'].agg(['min', 'max']).reset_index()
    sales = sales[(sales['min'] >= '2019-01-01') & (sales['max'] <= '2019-03-31')]
    df = pd.merge(sales, product, how='left', on='product_id')[['product_id', 'product_name']]
    return df

In [ ]:
sales_analysis(product, sales)

,product_id,product_name
0,1,S8
1,5,NaN


# 511 Анализ игрового процесса I
найти дату первого входа в систему для каждого игрока.


In [ ]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})

In [ ]:
activity.to_sql('activity',con,index=False,if_exists='replace')

5

In [ ]:
sql ='''
SELECT *
from activity

'''

In [ ]:
select(sql)

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01 00:00:00,5
1,1,2,2016-05-02 00:00:00,6
2,2,3,2017-06-25 00:00:00,1
3,3,1,2016-03-02 00:00:00,0
4,3,4,2018-07-03 00:00:00,5


In [ ]:
sql ='''
SELECT player_id, MIN(event_date) as first_login
FROM activity
GROUP BY player_id
'''


In [ ]:
select(sql)

,player_id,first_login
0,1,2016-03-01 00:00:00
1,2,2017-06-25 00:00:00
2,3,2016-03-02 00:00:00


In [ ]:
def game_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    t = activity.groupby('player_id')['event_date'].agg('min').reset_index()
    t = t.rename(columns={'event_date': 'first_login'})
    return t

In [ ]:
game_analysis(activity)

,player_id,first_login
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


In [ ]:
def game_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    return activity.groupby('player_id').agg(first_login=('event_date', 'min')).reset_index()

# 577 Бонус для сотрудников
Напишите решение, чтобы указать имя и сумму бонуса каждого сотрудника с бонусом менее 1000


In [ ]:
data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype({'empId':'Int64', 'name':'object', 'supervisor':'Int64', 'salary':'Int64'})
data = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data, columns=['empId', 'bonus1']).astype({'empId':'Int64', 'bonus1':'Int64'})

In [ ]:
employee.to_sql('employee',con,index=False,if_exists='replace')
bonus.to_sql('bonus',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
SELECT name, bonus
FROM employee LEFT JOIN bonus USING(empId)
WHERE bonus IS NULL OR bonus < 1000
'''

In [ ]:
select(sql)

,name,bonus
0,Brad,NaN
1,John,NaN
2,Dan,500.0


In [ ]:
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    t = pd.merge(employee, bonus, how='left', on='empId')
    t = t[(t['bonus1'] < 1000) | (t['bonus1'].isnull())]
    t = t.rename(columns={'bonus1': 'bonus'})
    return t[['name','bonus']]

In [ ]:
employee_bonus(employee,bonus)

,name,bonus
0,Brad,<NA>
1,John,<NA>
2,Dan,500


In [ ]:
# Вариант в одну строку
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    return employee.merge(bonus, how = "left").query("(bonus < 1000) | bonus.isna()")[["name","bonus"]]

# Решить на питоне


# 586 Клиент, разместивший наибольшее количество заказов
Напишите решение, чтобы найти customer_number для клиента, который разместил наибольшее количество заказов.

Тестовые примеры генерируются таким образом, что ровно один клиент разместил больше заказов, чем любой другой клиент.

In [ ]:
data = [[1, 1], [2, 2], [3, 3], [4, 3]]
orders = pd.DataFrame(data, columns=['order_number', 'customer_number']).astype({'order_number':'Int64', 'customer_number':'Int64'})

In [ ]:
orders.to_sql('orders',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
select customer_number
from (
SELECT customer_number, count(order_number) as cnt
from orders
group by customer_number) t
order by cnt desc
limit 1
'''

# select customer_number
# from Orders
# group by customer_number
# order by count(customer_number) desc limit 1;


In [ ]:
select(sql)

,customer_number
0,3


# 596 В Классах более 5 Учеников
Напишите решение, чтобы найти все классы, в которых учится по крайней мере пять учеников

In [ ]:
data = [['A', 'Math'], ['B', 'English'], ['C', 'Math'], ['D', 'Biology'], ['E', 'Math'], ['F', 'Computer'], ['G', 'Math'], ['H', 'Math'], ['I', 'Math']]
courses = pd.DataFrame(data, columns=['student', 'class']).astype({'student':'object', 'class':'object'})

In [ ]:
courses.to_sql('courses',con,index=False,if_exists='replace')

9

In [ ]:
sql ='''
select class
from courses
group by class
having count(student) >= 5
'''

In [ ]:
select(sql)

,class
0,Math


# 610 Суждение о треугольнике
Сообщите для каждых трех отрезков прямой, могут ли они образовывать треугольник.

In [ ]:
data = [[13, 15, 30], [10, 20, 15]]
triangle = pd.DataFrame(data, columns=['x', 'y', 'z']).astype({'x':'Int64', 'y':'Int64', 'z':'Int64'})

In [ ]:
triangle.to_sql('triangle',con,index=False,if_exists='replace')

2

In [ ]:
sql ='''
select *
from triangle
'''

In [ ]:
select(sql)

,x,y,z
0,13,15,30
1,10,20,15


In [ ]:
sql ='''
select x,y,z,
case when (x+y) > z and (x+z) > y and (y+z) > x then 'Yes' else 'No'
end as triangle
from Triangle
'''

In [ ]:
select(sql)

,x,y,z,triangle
0,13,15,30,No
1,10,20,15,Yes


# 1141 Активность пользователей за последние 30 дней Я
Напиши решение для определения ежедневного количества активных пользователей за период в 30 дней, заканчивающийся 2019-07-27 включительно. Пользователь был активен в someday, если он выполнил хотя бы одно действие в этот день.

In [ ]:
data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'], [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'], [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'], [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype({'user_id':'Int64', 'session_id':'Int64', 'activity_date':'datetime64[ns]', 'activity_type':'object'})

In [ ]:
activity.to_sql('activity',con,index=False,if_exists='replace')

11

In [ ]:
sql ='''
SELECT activity_date AS day, COUNT(DISTINCT user_id) AS active_users
FROM activity
GROUP BY activity_date
HAVING activity_date >= DATE_SUB('2019-07-27', INTERVAL 30 DAY)
'''
# Простое, хочу чтобы учитывало только одну дату
# select distinct activity_date as day,count(distinct user_id)  as active_users
# from Activity
# where activity_date between '2019-06-28' and '2019-07-27'
# group by activity_date

# 1148 Просмотры статей I

Напишите решение для поиска всех авторов, которые просмотрели хотя бы одну из своих собственных статей.

In [ ]:
data = [[1, 3, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})

In [ ]:
views.to_sql('views',con,index=False,if_exists='replace')

7

In [ ]:
sql ='''
select distinct author_id as id
from views
where viewer_id = author_id
'''

In [ ]:
select(sql)

,id
0,7
1,4


# 1179 Переформатируйте таблицу отделов

Переформатируйте таблицу таким образом, чтобы в ней были столбец идентификатора отдела и столбец выручки за каждый месяц.

In [ ]:
data = [[1, 8000, 'Jan'], [2, 9000, 'Jan'], [3, 10000, 'Feb'], [1, 7000, 'Feb'], [1, 6000, 'Mar']]
department = pd.DataFrame(data, columns=['id', 'revenue', 'month']).astype({'id':'Int64', 'revenue':'Int64', 'month':'object'})

In [ ]:
department.to_sql('department',con,index=False,if_exists='replace')

5

In [ ]:
# Решение взял, не знал, что так оно решается
sql ='''
SELECT DISTINCT
id,
SUM(CASE WHEN month = "Jan" THEN revenue END) AS Jan_Revenue,
SUM(CASE WHEN month = "Feb" THEN revenue END) AS Feb_Revenue,
SUM(CASE WHEN month = "Mar" THEN revenue END) AS Mar_Revenue,
SUM(CASE WHEN month = "Apr" THEN revenue END) AS Apr_Revenue,
SUM(CASE WHEN month = "May" THEN revenue END) AS May_Revenue,
SUM(CASE WHEN month = "Jun" THEN revenue END) AS Jun_Revenue,
SUM(CASE WHEN month = "Jul" THEN revenue END) AS Jul_Revenue,
SUM(CASE WHEN month = "Aug" THEN revenue END) AS Aug_Revenue,
SUM(CASE WHEN month = "Sep" THEN revenue END) AS Sep_Revenue,
SUM(CASE WHEN month = "Oct" THEN revenue END) AS Oct_Revenue,
SUM(CASE WHEN month = "Nov" THEN revenue END) AS Nov_Revenue,
SUM(CASE WHEN month = "Dec" THEN revenue END) AS Dec_Revenue
FROM Department
GROUP BY id

'''

In [ ]:
select(sql)

DatabaseError: Execution failed on sql '
SELECT id,
       SUM(if(month = 'Jan', revenue, null)) AS Jan_Revenue,
       SUM(if(month = 'Feb', revenue, null)) AS Feb_Revenue,
       SUM(if(month = 'Mar', revenue, null)) AS Mar_Revenue,
       SUM(if(month = 'Apr', revenue, null)) AS Apr_Revenue,
       SUM(if(month = 'May', revenue, null)) AS May_Revenue,
       SUM(if(month = 'Jun', revenue, null)) AS Jun_Revenue,
       SUM(if(month = 'Jul', revenue, null)) AS Jul_Revenue,
       SUM(if(month = 'Aug', revenue, null)) AS Aug_Revenue,
       SUM(if(month = 'Sep', revenue, null)) AS Sep_Revenue,
       SUM(if(month = 'Oct', revenue, null)) AS Oct_Revenue,
       SUM(if(month = 'Nov', revenue, null)) AS Nov_Revenue,
       SUM(if(month = 'Dec', revenue, null)) AS Dec_Revenue
FROM Department
GROUP BY id
': no such function: if

# 1251 Средняя цена продажи

Напишите решение, чтобы найти среднюю цену продажи для каждого продукта. средняя цена должна быть округлена до 2 знаков после запятой.

In [ ]:
data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})

In [ ]:
prices.to_sql('prices',con,index=False,if_exists='replace')
units_sold.to_sql('units_sold',con,index=False,if_exists='replace')

4

In [ ]:
sql ='''
select prices.product_id, ROUND(SUM(units_sold.units * prices.price) / SUM(units_sold.units), 2) AS average_price
from prices left join units_sold ON prices.product_id = units_sold.product_id AND units_sold.purchase_date BETWEEN prices.start_date AND prices.end_date
group by prices.product_id
'''

In [ ]:
select(sql)

,product_id,average_price
0,1,6
1,2,16


# 1327 Перечислите товары, заказанные за определенный период

Напишите решение, чтобы получить названия товаров, у которых в феврале 2020 года было заказано не менее 100 единиц, и их количество.

In [ ]:
data = [[1, 'Leetcode Solutions', 'Book'], [2, 'Jewels of Stringology', 'Book'], [3, 'HP', 'Laptop'], [4, 'Lenovo', 'Laptop'], [5, 'Leetcode Kit', 'T-shirt']]
products = pd.DataFrame(data, columns=['product_id', 'product_name', 'product_category']).astype({'product_id':'Int64', 'product_name':'object', 'product_category':'object'})
data = [[1, '2020-02-05', 60], [1, '2020-02-10', 70], [2, '2020-01-18', 30], [2, '2020-02-11', 80], [3, '2020-02-17', 2], [3, '2020-02-24', 3], [4, '2020-03-01', 20], [4, '2020-03-04', 30], [4, '2020-03-04', 60], [5, '2020-02-25', 50], [5, '2020-02-27', 50], [5, '2020-03-01', 50]]
orders = pd.DataFrame(data, columns=['product_id', 'order_date', 'unit']).astype({'product_id':'Int64', 'order_date':'datetime64[ns]', 'unit':'Int64'})

In [ ]:
products.to_sql('products',con,index=False,if_exists='replace')
orders.to_sql('orders',con,index=False,if_exists='replace')

12

In [ ]:
sql ='''
select *
from products p inner join orders o ON p.product_id = o.product_id AND order_date BETWEEN '2020-02-01' AND '2020-02-28'
'''

In [ ]:
select(sql)

,product_id,product_name,product_category,product_id,order_date,unit
0,1,Leetcode Solutions,Book,1,2020-02-05 00:00:00,60
1,1,Leetcode Solutions,Book,1,2020-02-10 00:00:00,70
2,2,Jewels of Stringology,Book,2,2020-02-11 00:00:00,80
3,3,HP,Laptop,3,2020-02-17 00:00:00,2
4,3,HP,Laptop,3,2020-02-24 00:00:00,3
5,5,Leetcode Kit,T-shirt,5,2020-02-25 00:00:00,50
6,5,Leetcode Kit,T-shirt,5,2020-02-27 00:00:00,50


In [ ]:
sql ='''
select p.product_name, SUM(o.unit) unit
from products p inner join orders o ON p.product_id = o.product_id AND order_date BETWEEN '2020-02-01' AND '2020-02-28'
group by p.product_name
HAVING SUM(o.unit) >= 100
'''

In [ ]:
select(sql)

,product_name,unit
0,Leetcode Kit,100
1,Leetcode Solutions,130


# 1484 Сгруппируйте проданные товары по дате

Напишите решение, чтобы найти для каждой даты количество проданных различных товаров и их названия.

Названия проданных товаров для каждой даты должны быть отсортированы лексикографически.

Верните таблицу результатов, упорядоченную по sell_date.

In [ ]:
data = [['2020-05-30', 'Headphone'], ['2020-06-01', 'Pencil'], ['2020-06-02', 'Mask'], ['2020-05-30', 'Basketball'], ['2020-06-01', 'Bible'], ['2020-06-02', 'Mask'], ['2020-05-30', 'T-Shirt']]
activities = pd.DataFrame(data, columns=['sell_date', 'product']).astype({'sell_date':'datetime64[ns]', 'product':'object'})

In [ ]:
activities.to_sql('activities',con,index=False,if_exists='replace')


7

In [ ]:
sql ='''
select sell_date, count(distinct product) as num_sold, GROUP_CONCAT(DISTINCT product) AS products
from activities
group by sell_date
'''

In [ ]:
# для постгреса
sql ='''
SELECT sell_date, COUNT(DISTINCT product) AS num_sold , STRING_AGG(DISTINCT product, ',' ORDER BY product) AS products
FROM activities
GROUP BY sell_date
'''

In [ ]:
select(sql)

,sell_date,num_sold,products
0,2020-05-30 00:00:00,3,"Headphone,Basketball,T-Shirt"
1,2020-06-01 00:00:00,2,"Pencil,Bible"
2,2020-06-02 00:00:00,1,Mask


# 1517 Найдите пользователей с действительными электронными письмами

Напишите решение для поиска пользователей, у которых есть действительные электронные письма.

Действительное электронное письмо имеет префикс имени и домен, в котором:

Имя префикса представляет собой строку, которая может содержать буквы (прописные или строчные), цифры, символ подчеркивания "_", точку "." и/или тире "-". Имя префикса должно начинаться с буквы.
Домен - "@leetcode.com".

In [ ]:
data = [[1, 'Winston', 'winston@leetcode.com'], [2, 'Jonathan', 'jonathanisgreat'], [3, 'Annabelle', 'bella-@leetcode.com'], [4, 'Sally', 'sally.come@leetcode.com'], [5, 'Marwan', 'quarz#2020@leetcode.com'], [6, 'David', 'david69@gmail.com'], [7, 'Shapiro', '.shapo@leetcode.com']]
users = pd.DataFrame(data, columns=['user_id', 'name', 'mail']).astype({'user_id':'int64', 'name':'object', 'mail':'object'})

In [ ]:
users.to_sql('users',con,index=False,if_exists='replace')

7

In [ ]:
sql ='''
SELECT user_id, name, mail
FROM users
WHERE mail LIKE '[a-zA-Z]%@leetcode.com' AND LEFT(mail, LEN(mail) - 13) NOT LIKE '%[^0-9a-zA-Z_.-]%';
'''

# 2356 Количество уникальных предметов, преподаваемых каждым преподавателем

Напишите решение для вычисления количества уникальных предметов, которые преподает каждый преподаватель в университете.

In [ ]:
data = [[1, 2, 3], [1, 2, 4], [1, 3, 3], [2, 1, 1], [2, 2, 1], [2, 3, 1], [2, 4, 1]]
teacher = pd.DataFrame(data, columns=['teacher_id', 'subject_id', 'dept_id']).astype({'teacher_id':'Int64', 'subject_id':'Int64', 'dept_id':'Int64'})

In [ ]:
teacher.to_sql('teacher',con,index=False,if_exists='replace')

7

In [ ]:
sql ='''
SELECT teacher_id, count(distinct subject_id) as cnt
from teacher
group by teacher_id
'''

In [ ]:
select(sql)

,teacher_id,cnt
0,1,2
1,2,4


# 1965 Сотрудники с отсутствующей информацией

Напишите решение, позволяющее сообщать идентификаторы всех сотрудников с отсутствующей информацией. Информация о сотруднике отсутствует, если:

Имя сотрудника отсутствует или Зарплата сотрудника отсутствует.

Возвращает таблицу результатов, упорядоченную по employee_id в порядке возрастания.

In [ ]:
data = [[2, 'Crew'], [4, 'Haven'], [5, 'Kristian']]
employees = pd.DataFrame(data, columns=['employee_id', 'name']).astype({'employee_id':'Int64', 'name':'object'})
data = [[5, 76071], [1, 22517], [4, 63539]]
salaries = pd.DataFrame(data, columns=['employee_id', 'salary']).astype({'employee_id':'Int64', 'salary':'Int64'})

In [ ]:
employees.to_sql('employees',con,index=False,if_exists='replace')
salaries.to_sql('salaries',con,index=False,if_exists='replace')

3

In [ ]:
employees

,employee_id,name
0,2,Crew
1,4,Haven
2,5,Kristian


In [ ]:
salaries

,employee_id,salary
0,5,76071
1,1,22517
2,4,63539


In [ ]:
sql ='''
select employee_id
from (
SELECT employee_id, name, salary
from employees
LEFT JOIN salaries using(employee_id)
UNION ALL
SELECT employee_id, name, salary
from salaries
LEFT JOIN employees using(employee_id))
where name is null or salary is null
order by employee_id
'''

In [ ]:
select(sql)

,employee_id,name,salary
0,2,Crew,None
1,1,22517,None


# 1934 Очки подтверждения

Коэффициент подтверждения пользователя - это количество "подтвержденных" сообщений, деленное на общее количество запрошенных сообщений с подтверждением. Коэффициент подтверждения пользователя, который не запрашивал никаких сообщений с подтверждением, равен 0. Округлите коэффициент подтверждения до двух знаков после запятой.

Напишите решение, чтобы найти частоту подтверждений каждого пользователя.

Верните таблицу результатов в любом порядке.

Формат результата приведен в следующем примере.

In [ ]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})

In [ ]:
signups.to_sql('signups',con,index=False,if_exists='replace')
confirmations.to_sql('confirmations',con,index=False,if_exists='replace')

7

In [ ]:
signups

,user_id,time_stamp
0,3,2020-03-21 10:16:13
1,7,2020-01-04 13:57:59
2,2,2020-07-29 23:09:44
3,6,2020-12-09 10:39:37


In [ ]:
confirmations

,user_id,time_stamp,action
0,3,2021-01-06 03:30:46,timeout
1,3,2021-07-14 14:00:00,timeout
2,7,2021-06-12 11:57:29,confirmed
3,7,2021-06-13 12:58:28,confirmed
4,7,2021-06-14 13:59:27,confirmed
5,2,2021-01-22 00:00:00,confirmed
6,2,2021-02-28 23:59:59,timeout


In [ ]:
sql ='''
SELECT s.user_id, ROUND(AVG(CASE WHEN c.action = 'confirmed' THEN 1 ELSE 0 END), 2) AS confirmation_rate
FROM Signups s
LEFT JOIN Confirmations c ON s.user_id = c.user_id
GROUP BY s.user_id
'''

In [ ]:
select(sql)

,user_id,confirmation_rate
0,2,0.5
1,3,0.0
2,6,0.0
3,7,1.0


# 1978 Сотрудники, чей менеджер покинул компанию

Найдите идентификаторы сотрудников, чья зарплата строго меньше 30000 долларов и чей менеджер покинул компанию. Когда менеджер покидает компанию, его информация удаляется из таблицы Employees, но в отчетах по-прежнему указывается manager_id ушедшего менеджера.

Возвращает таблицу результатов, упорядоченную по employee_id.

In [ ]:
data = [[3, 'Mila', 9, 60301], [12, 'Antonella', None, 31000], [13, 'Emery', None, 67084], [1, 'Kalel', 11, 21241], [9, 'Mikaela', None, 50937], [11, 'Joziah', 6, 28485]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'manager_id', 'salary']).astype({'employee_id':'Int64', 'name':'object', 'manager_id':'Int64', 'salary':'Int64'})

In [ ]:
employees.to_sql('employees',con,index=False,if_exists='replace')

6

In [ ]:
employees

,employee_id,name,manager_id,salary
0,3,Mila,9,60301
1,12,Antonella,<NA>,31000
2,13,Emery,<NA>,67084
3,1,Kalel,11,21241
4,9,Mikaela,<NA>,50937
5,11,Joziah,6,28485


In [ ]:
sql ='''
select e.employee_id
from employees e left join employees em on e.manager_id = em.employee_id
where e.salary < 30000 and em.employee_id is null and e.manager_id is not null
order by e.employee_id
'''

In [ ]:
select(sql)

,employee_id
0,11


In [ ]:
sql ='''
SELECT employee_id
FROM Employees
WHERE salary < 30000
AND manager_id NOT IN (
  SELECT employee_id FROM Employees
)
ORDER BY employee_id
'''


# 1907 Подсчитайте категории зарплат

Напишите решение для расчета количества банковских счетов для каждой категории зарплат. Категории зарплат следующие:

"Низкая зарплата": все зарплаты строго меньше 20000 долларов.
"Средняя зарплата": Все зарплаты в диапазоне включительно [20000 долларов, 50000 долларов].
"Высокая зарплата": Все зарплаты, строго превышающие 50000 долларов.
Таблица результатов должна содержать все три категории. Если в категории нет учетных записей, верните 0

In [ ]:
data = [[3, 108939], [2, 12747], [8, 87709], [6, 91796]]
accounts = pd.DataFrame(data, columns=['account_id', 'income']).astype({'account_id':'Int64', 'income':'Int64'})

In [ ]:
accounts.to_sql('accounts',con,index=False,if_exists='replace')

4

In [ ]:
accounts

,account_id,income
0,3,108939
1,2,12747
2,8,87709
3,6,91796


In [ ]:
# Много вариантов испробовал через COALESCE и CASE, суть в том, что COALESCE заменяет нулы на ноли, но в изначальной выборке, у нас просто нет нулов. Нет значений. Получается, самое оптимальное, это как в варианте ниже

sql ='''
SELECT "Low Salary" AS category,
       sum(income < 20000) AS accounts_count
  FROM Accounts

UNION

SELECT "Average Salary" AS category,
       sum(income BETWEEN 20000 AND 50000) AS accounts_count
  FROM Accounts

UNION

SELECT "High Salary" AS category,
       sum(income > 50000) AS accounts_count
  FROM Accounts;
'''

In [ ]:
select(sql)

,category,accounts_count
0,Average Salary,0
1,High Salary,3
2,Low Salary,1
